In [1]:
import numpy as np
import time
import random
from itertools import combinations

In [3]:
start_time = time.time()

In [5]:
# загружаем датасет 

from sklearn.datasets import load_digits
dataset = load_digits()

train = np.zeros(shape = (dataset.data.shape[0], dataset.data.shape[1]), dtype = bool)
y_train = np.zeros(dataset.data.shape[0])
for i in range(dataset.data.shape[0]):
    image = dataset.data[i]
    image = np.array(image)
    binary_image = (image > 8).astype(bool)      
    train[i] = binary_image
    y_train[i] = dataset.target[i]

#разбиваем на классы
number = {}
for i in range(10):
    number[i] = train[y_train == i]

#делаем классы уникальными 
for i in range(10):
    number[i] = np.unique(number[i], return_counts = False, axis = 0)

#делим на train и test
number_train = {}
number_test = {}
first_dimensions = np.array([arr.shape[0] for key, arr in number.items()])
p = round(np.sum(first_dimensions) * 0.2 / 10)  # выделаем 20 процентов от всей выборки на тестовую выборку
print("количество объектов в обучающей и тестовой выборках:")
for i in range(10):
    length = number[i].shape[0]
    number_train[i] = number[i][: length - p, :]
    number_test[i] = number[i][length - p : length, :]
    print(i, ": ", number_train[i].shape[0], " + ", number_test[i].shape[0], " = ", number_train[i].shape[0] + number_test[i].shape[0])


количество объектов в обучающей и тестовой выборках:
0 :  133  +  35  =  168
1 :  122  +  35  =  157
2 :  142  +  35  =  177
3 :  147  +  35  =  182
4 :  144  +  35  =  179
5 :  145  +  35  =  180
6 :  142  +  35  =  177
7 :  143  +  35  =  178
8 :  139  +  35  =  174
9 :  145  +  35  =  180


In [6]:
#строим матрицу C_nmL
mas = np.zeros(train.shape[1], dtype=bool)
for i in range(0, 9):
    for j in range(i+1, 10):
        mas_xor = np.bitwise_xor(number_train[i][:, np.newaxis, :], number_train[j][np.newaxis, :, :])
        mas_xor = mas_xor.reshape(-1, number_train[0].shape[1])
        mas = np.vstack((mas, mas_xor))
        mas = np.unique(mas, return_counts = False, axis = 0)

# удаляем первый ненужный элемент
maska = np.ones(mas.shape[0], dtype=bool)
maska[0] = False
mas = mas[maska]
print("C_nmL ", mas.shape)

C_nmL  (882785, 64)


In [8]:
# удаляем старший из пары
bit_mas = mas.astype(np.uint8)   # массив в 0 и 1
powers = 1 << np.arange(63, -1, -1, dtype=np.uint64)    # веткор весов каждого разряда
result = np.dot(bit_mas, powers)   # массив из чисел в 10 системе 
to_remove = []  # для чисел из пары, которые мы удаляем
mask = np.zeros(shape = (64, 64), dtype = bool)
for i in range(64):
    mask[i][i] = True
    
for i in range(mas.shape[0]):
    bit_x = mas[i]
    chislo_x = result[i]
    neighbors = np.bitwise_xor(bit_x, mask)   # True-False-битовое представление соседей
    neighbors = neighbors.astype(np.uint8)    # 0-1-битовое представление соседей
    sosedy = np.dot(neighbors, powers)    # числовое представление соседей
    for j in range(neighbors.shape[0]):
        if sosedy[j] in result:
            if sosedy[j] > chislo_x and sosedy[j] not in to_remove:
                to_remove.append(sosedy[j])
            elif sosedy[j] < chislo_x and chislo_x not in to_remove:
                to_remove.append(chislo_x)
new_mask = np.isin(result, to_remove, invert=True)
otvet = result[new_mask]
powers = 1 << np.arange(63, -1, -1, dtype=np.uint64) 
bit_otvet = ((otvet[:, None] & powers) > 0)
bit_otvet.shape

(699656, 64)

In [43]:
# удаляем нулевые столбцы, которые точно не войдут в тест
vector = bit_otvet.sum(axis=0)
stolbs = np.where(vector == 0)
stolbs = np.array(stolbs)
new_mas = np.delete(bit_otvet, stolbs, axis=1)
vector = new_mas.sum(axis=0)
print("C_nmL после удаления нулевых столбцов: ", new_mas.shape)
print("т.е. удалено ", stolbs.shape[1], " столбцов")

C_nmL после удаления нулевых столбцов:  (699656, 50)
т.е. удалено  14  столбцов


In [10]:
# находим первые 15 столбцов с максимальным количеством единиц, которые будем удалять
num = 0
m, n = new_mas.shape
uncovered_rows = set(range(m))
selected_columns = []
while uncovered_rows and num != 15:
    max_cover = -1
    best_col = -1
    for j in range(n):
        covered_rows = set(np.where(new_mas[:, j] == 1)[0])
        new_coverage = uncovered_rows.intersection(covered_rows)
        if len(new_coverage) > max_cover:
            max_cover = len(new_coverage)
            best_col = j
    if best_col == -1:
        print("Не удалось покрыть оставшиеся строки\n")
        break
    selected_columns.append(best_col)
    num += 1
    covered_rows = set(np.where(new_mas[:, best_col] == 1)[0])
    uncovered_rows -= covered_rows
    if len(uncovered_rows) == 0:
        break


selected_columns = np.sort(np.array(selected_columns))
print("\nвыбранные строки: ", selected_columns)

covered_rows = set(range(m))
covered_rows -= uncovered_rows
covered_rows = np.array(list(covered_rows))


выбранные строки:  [ 7 14 15 16 17 20 21 23 27 29 32 33 34 41 47]


In [11]:
# Удаляем строки
arr_new = np.delete(new_mas, covered_rows, axis=0)
# Удаляем столбцы
arr_new = np.delete(arr_new, selected_columns, axis=1)
print("остаток от C_nmL ",arr_new.shape)

остаток от C_nmL  (14, 35)


In [12]:
# перебором находим остаток покрытия
m, n = arr_new.shape
columns = range(n)
all_solutions = []
for t in range(1, 10):
    for J in combinations(columns, t):
        sel_mas = arr_new[:, J]
        row_sums = sel_mas.sum(axis=1)
        has_zero = np.any(row_sums == 0)        # значение haz_zero: Fasle = нет 0 -> это покрытие 
                                                                   # True = усть 0 -> это НЕ покрытие 
        if not has_zero :
            J_set = set(J)
            flag = True
            for k in range(len(all_solutions)):
                k_set = set(all_solutions[k])
                if k_set.issubset(J_set):
                    flag = False
            if flag:
                J_list = list(J)
                J_list = sorted(J_list)
                all_solutions.append(J_list)    

print("всего: ", len(all_solutions))


всего:  1156


In [13]:
#сохраняем переборные части теста 
all_solution = np.array(all_solutions, dtype="object")
np.save('perebor_part.npy', all_solution)

#загружаем назад, если до этого устали 
all_solutions = np.load('perebor_part.npy', allow_pickle=True)
for i in range(len(all_solutions)):
    all_solutions[i] = list(all_solutions[i])


In [15]:
#создаем бинарную матрицу тестов, где True стоит в столбцах, которые входят в тест
dim = train[0].shape[0] - stolbs.shape[1] - selected_columns.shape[0]
matrix = np.zeros(shape = (all_solutions.shape[0], dim))
for i in range(all_solutions.shape[0]):
    matrix[i][all_solutions[i]] = True

# вставляем столбцы из 1 на позиции selected_columns
for i in range(selected_columns.shape[0]):
    matrix = np.insert(matrix, selected_columns[i], 1, axis=1)

test = np.zeros(shape = (all_solutions.shape[0], train[0].shape[0]))   
full_range = np.arange(train[0].shape[0])
dop_stolbs = np.setdiff1d(full_range, stolbs)
test[:, dop_stolbs] = matrix
test = test.astype(bool)

# проверка
flag = True
for i in range(test.shape[0]):
    ind = test[i]
    sel_mas = mas[:, ind]
    vector = sel_mas.sum(axis = 1)
    has_zero = np.any(vector == 0)
    if has_zero :
        flag = False
flag

# True - это тесты
# False - это НЕ тесты

True

In [16]:
# сохраняем финальный вариант тестов
solution = np.array(test)
np.save('test.npy', solution)

In [17]:
# Функция для определения класса для одного тестового объекта
def classify_sample(sample, train_data):
    min_distance = np.inf
    predicted_class = None
    for class_label, class_samples in train_data.items():
        distances = np.linalg.norm(np.bitwise_xor(class_samples, sample), axis=1)
        mean_distance = np.mean(distances)
        if mean_distance < min_distance:
            min_distance = mean_distance
            predicted_class = class_label
    return predicted_class


In [18]:
# Классификация всей тестовой выборки
y_pred = []
for class_label, samples in number_test.items():
    for i, sample in enumerate(samples):
        predicted = classify_sample(sample, number_train)
        y_pred.append(predicted)
        

In [19]:
# метрика качества
num = np.arange(10)
y_true = np.repeat(num, number_test[0].shape[0])
print("accuracy 2 = ", np.sum(y_true == y_pred) / y_true.shape[0])

accuracy =  0.7342857142857143


In [20]:
end_time = time.time() 
execution_time = end_time - start_time
print(f"Время выполнения программы: {execution_time} секунд")

Время выполнения программы: 42715.38802289963 секунд


In [35]:
#находим метрику качества вторым способом
num = np.arange(10)
y_true = np.repeat(num, number_test[0].shape[0])
print("accuracy 2 = ", np.sum(y_true == y_pred) / y_true.shape[0])

accuracy 2 =  0.7342857142857143


In [39]:
#находим метрику качества первым способом
y_pred = np.zeros(number_test[0].shape[0] * 10)
num = np.arange(10)
y_true = np.repeat(num, number_test[0].shape[0])
for i in range(10):
    for j in range(number_test[i].shape[0]):
        vector =  np.zeros(10)
        for k in range(test.shape[0]):
            maska = number_test[i][j][test[k]]
            for h in range(10):
                part = number_train[h][:, test[k]]
                result = np.any(np.all(part == maska, axis=1))
                if result:
                    vector[h] += 1
        for h in range(10):
            vector[h] /= number_train[h].shape[0]
        mx = np.max(vector)
        count_max = np.sum(vector == mx)
        poz = np.argmax(vector)
        if count_max == 1 and poz == i:
            y_pred[i*number_test[0].shape[0] + j] = i
        elif mx == i:
            y_pred[i*number_test[0].shape[0] + j] = i
        else:
            y_pred[i*number_test[0].shape[0] + j] = poz
print("accuracy 1 = ", np.sum(y_true == y_pred) / y_true.shape[0])

accuracy 1 =  0.4657142857142857
